<a href="https://colab.research.google.com/github/Avina20/DocsChatbot/blob/main/doc_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install langchain langchain-community sentence-transformers faiss-cpu python-dotenv google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1

In [2]:
import os
import requests
import json
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_community.document_loaders import TextLoader
from langchain.schema import Document
from langchain.llms.base import LLM
from langchain_community.embeddings import HuggingFaceEmbeddings
from typing import Optional, List, Any

In [15]:
def test_direct_gemini():
    """Test direct Gemini API call"""
    GOOGLE_API_KEY = ""
    prompt = 'what is java'
    url = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent"

    headers = {
        'Content-Type': 'application/json',
        'X-goog-api-key': GOOGLE_API_KEY
    }

    payload = {
        "contents": [
            {
                "parts": [
                    {
                        "text": prompt
                    }
                ]
            }
        ]
    }

    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        result = response.json()
        print("Direct API test:")
        print(result['candidates'][0]['content']['parts'][0]['text'])
    except Exception as e:
        print(f"Direct API test failed: {e}")

In [16]:
test_direct_gemini()

Direct API test:
Java is a high-level, class-based, object-oriented programming language that is designed to have as few implementation dependencies as possible. It is a general-purpose programming language intended to let application developers **write once, run anywhere (WORA)**, meaning that compiled Java code can run on all platforms that support Java without the need for recompilation.

Here's a breakdown of the key aspects of Java:

**Key Features and Characteristics:**

*   **Object-Oriented Programming (OOP):** Java is built around the concepts of objects, classes, inheritance, polymorphism, and encapsulation. This allows for code that is modular, reusable, and easier to maintain.

*   **Platform Independence (WORA):**  Java achieves platform independence through the **Java Virtual Machine (JVM)**.  Java code is compiled into bytecode, which is then executed by the JVM.  The JVM acts as an intermediary between the compiled code and the underlying operating system, making the co

In [8]:
class GeminiLLM(LLM):
    """Custom Gemini LLM wrapper using direct API calls"""

    # Declare Pydantic fields
    api_key: str
    model: str = "gemini-2.0-flash"
    temperature: float = 0.7
    max_tokens: int = 1000
    base_url: str = "https://generativelanguage.googleapis.com/v1beta/models"

    def __init__(self, api_key: str, **kwargs):
        super().__init__(api_key=api_key, **kwargs)

    @property
    def _llm_type(self) -> str:
        return "gemini"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        """Make API call to Gemini"""
        try:
            url = f"{self.base_url}/{self.model}:generateContent"

            headers = {
                'Content-Type': 'application/json',
                'X-goog-api-key': self.api_key
            }

            payload = {
                "contents": [
                    {
                        "parts": [
                            {
                                "text": prompt
                            }
                        ]
                    }
                ],
                "generationConfig": {
                    "temperature": self.temperature,
                    "maxOutputTokens": self.max_tokens
                }
            }

            response = requests.post(url, headers=headers, json=payload)
            response.raise_for_status()

            result = response.json()

            if 'candidates' in result and len(result['candidates']) > 0:
                return result['candidates'][0]['content']['parts'][0]['text']
            else:
                return "No response generated"

        except requests.exceptions.RequestException as e:
            return f"API Error: {str(e)}"
        except KeyError as e:
            return f"Response parsing error: {str(e)}"
        except Exception as e:
            return f"Unexpected error: {str(e)}"


In [4]:
class DocumentChatbot:
    def __init__(self, google_api_key):
        # Initialize Gemini with custom wrapper
        self.llm = GeminiLLM(api_key=google_api_key)

        # Use free HuggingFace embeddings (no API key needed)
        self.embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2"
        )

        # Initialize memory for conversation history
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True,
            output_key="answer"
        )

        self.vectorstore = None
        self.qa_chain = None

    def load_documents(self, file_paths):
        """Load and process documents"""
        documents = []

        for file_path in file_paths:
            try:
                if file_path.endswith('.txt'):
                    loader = TextLoader(file_path, encoding='utf-8')
                    docs = loader.load()
                    documents.extend(docs)
                else:
                    # Handle other file types or plain text
                    with open(file_path, 'r', encoding='utf-8') as f:
                        content = f.read()
                    doc = Document(page_content=content, metadata={"source": file_path})
                    documents.append(doc)
            except Exception as e:
                print(f"Error loading {file_path}: {e}")

        return documents

    def setup_vectorstore(self, documents):
        """Create vector store from documents"""
        if not documents:
            print("No documents to process")
            return

        # Split documents into chunks
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len
        )

        splits = text_splitter.split_documents(documents)

        if not splits:
            print("No text chunks created")
            return

        # Create vector store
        self.vectorstore = FAISS.from_documents(splits, self.embeddings)

        # Setup Q&A chain
        self.qa_chain = ConversationalRetrievalChain.from_llm(
            llm=self.llm,
            retriever=self.vectorstore.as_retriever(search_kwargs={"k": 3}),
            memory=self.memory,
            return_source_documents=True,
            verbose=True
        )

    def add_sample_documents(self):
        """Add some sample documents for testing"""
        sample_docs = [
            Document(
                page_content="Python is a high-level programming language known for its simplicity and readability. It was created by Guido van Rossum and first released in 1991. Python supports multiple programming paradigms including procedural, object-oriented, and functional programming.",
                metadata={"source": "python_intro.txt"}
            ),
            Document(
                page_content="LangChain is a framework for developing applications powered by language models. It provides tools for chaining together different components to build complex AI applications. LangChain supports various LLMs and provides abstractions for common tasks.",
                metadata={"source": "langchain_info.txt"}
            ),
            Document(
                page_content="Gemini 2.0 Flash is Google's latest AI model, optimized for speed and efficiency while maintaining high quality responses. It excels at reasoning, coding, and multimodal understanding. The model supports both text and image inputs.",
                metadata={"source": "gemini_info.txt"}
            )
        ]

        self.setup_vectorstore(sample_docs)

    def chat(self, question):
        """Ask a question and get response with sources"""
        if not self.qa_chain:
            return {
                "answer": "Please load documents first using add_sample_documents() or load_documents()",
                "sources": []
            }

        try:
            result = self.qa_chain({"question": question})

            response = {
                "answer": result["answer"],
                "sources": [doc.metadata.get("source", "Unknown") for doc in result["source_documents"]]
            }

            return response
        except Exception as e:
            return {
                "answer": f"Error processing question: {str(e)}",
                "sources": []
            }

In [13]:
def main():
    # Replace with your actual API key
    GOOGLE_API_KEY = ""  # Note: Use environment variables in production!

    # Initialize chatbot
    print("Initializing chatbot...")
    chatbot = DocumentChatbot(GOOGLE_API_KEY)

    # Add sample documents (or load your own)
    print("Loading sample documents...")
    chatbot.add_sample_documents()

    print("\n🤖 Document Chatbot Ready!")
    print("Ask questions about the loaded documents. Type 'quit' to exit.\n")

    while True:
        question = input("You: ")
        if question.lower() in ['quit', 'exit', 'q']:
            break

        try:
            response = chatbot.chat(question)
            print(f"\nBot: {response['answer']}")
            if response['sources']:
                print(f"Sources: {', '.join(response['sources'])}\n")
            else:
                print()
        except Exception as e:
            print(f"Error: {e}\n")



In [17]:
main()

Initializing chatbot...
Loading sample documents...

🤖 Document Chatbot Ready!
Ask questions about the loaded documents. Type 'quit' to exit.

You: what is python


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Python is a high-level programming language known for its simplicity and readability. It was created by Guido van Rossum and first released in 1991. Python supports multiple programming paradigms including procedural, object-oriented, and functional programming.

LangChain is a framework for developing applications powered by language models. It provides tools for chaining together different components to build complex AI applications. LangChain supports various LLMs and provides abstractions for common tasks.

Gemini 2.0 Flash is Google's latest AI model, opti